In [10]:
import os
import requests
from IPython.display import Markdown, display, update_display
from bs4 import BeautifulSoup
from openai import OpenAI
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

# Connecting to Nebius LLM via API
Setup connection to Nebius API

In [29]:
#Find the key file

os.chdir("C:\\Users\\vital\\PythonStuff\\keys")
cwd = os.getcwd() 

with open("nebius_api_key", "r") as file:
    nebius_api_key = file.read().strip()

os.environ["NEBIUS_API_KEY"] = nebius_api_key

# Nebius uses the same OpenAI() class, but with additional details
nebius_client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

llama_8b_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
llama_70b_model ="meta-llama/Llama-3.3-70B-Instruct"
gemma_9b_model = "google/gemma-2-9b-it-fast"
Qwen2_5_72B_model = "Qwen/Qwen2.5-72B-Instruct"
DeepSeek_V33024 ="deepseek-ai/DeepSeek-V3-0324"
openai_20b = "openai/gpt-oss-20b"
Hermes_4_70B_model ="NousResearch/Hermes-4-70B"

In [55]:
#userPrompt = "How many words are there in your answer to this prompt?"
userPrompt = """A man has $84552 today. The annuity earns 4% annually. His age is 68 today.
If he choose not to go for "cpf life", he will get monthly $535 for 20 years.
If he choose to go for "cpf life", he will get monthly $470 every month for 31 years.
He decided to choose "cpf life", but live till 88 years old. How many % did he lose out?
"""

# Function to chat with LLM model

In [54]:
def answer_with_llm(prompt: str,
                    system_prompt="You are a helpful assistant",
                    max_tokens=1048,
                    client=nebius_client,
                    model=llama_8b_model,
                    prettify=True,
                    stream=False,
                    displayMode="markdown",
                    temperature=0.7) -> str:

    messages = []

    if system_prompt:
        messages.append(
            {
                "role": "system",
                "content": system_prompt
            }
        )

    messages.append(
        {
            "role": "user",
            "content": prompt
        }
    )
    if not stream:
        completion = client.chat.completions.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            temperature=temperature
        )
        return completion.choices[0].message.content

    # ✅ Streaming mode
    stream_resp = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature,
        stream=True
    )

    
    print(f"🔁 Streaming from model: {model}\n")
    full_text = ""

    # Only create Markdown display handle if needed
    md_handle = None
    if displayMode == "markdown":
        md_handle = display(Markdown(""), display_id=True)

    # ✅ Single loop
    for chunk in stream_resp:
        if chunk.choices and chunk.choices[0].delta.content:
            token = chunk.choices[0].delta.content
            full_text += token

            if displayMode == "markdown":
                update_display(Markdown(full_text), display_id=md_handle.display_id)
            elif displayMode == "console":
                print(token, end="", flush=True)

    print("\n✅ Done streaming.\n")
    return None
    
    '''
    full_text = ""
    print(f"🔁 Streaming from model: {model}\n")
    for chunk in stream_resp:
        if chunk.choices and chunk.choices[0].delta.content:
            token = chunk.choices[0].delta.content
            print(token, end="", flush=True)
            full_text += token

    print("\n✅ Done streaming.\n")
    return full_text
    '''
    #if prettify:
    #    return prettify_string(completion.choices[0].message.content)
   # else:
    
    

In [14]:
def display_answer(prompt,model,stream,displayMode):
    summary = answer_with_llm(prompt, model=model, stream=stream, displayMode=displayMode)
    if not stream:
        if displayMode == "markdown":
            display(Markdown(summary))
        else:
            print(summary)
    #display(Markdown(summary))

In [56]:
display_answer(userPrompt,llama_8b_model,True,"markdown");

🔁 Streaming from model: meta-llama/Meta-Llama-3.1-8B-Instruct



Let's break down the calculations:

**Scenario 1: Without "CPF Life"**

* Monthly payment: $535
* Duration: 20 years
* Total payments: $535 x 12 x 20 = $128,400

**Scenario 2: With "CPF Life"**

* Monthly payment: $470
* Duration: 31 years (since he lives till 88, we need to calculate the payments for 20 years after age 68, plus 11 years before that)
	+ First 20 years: $470 x 12 x 20 = $113,600
	+ Remaining 11 years: $470 x 12 x 11 = $61,324
	+ Total payments: $113,600 + $61,324 = $174,924

Since the man lives till 88, we need to calculate the present value of the payments for 20 years after age 68, and the payments for the remaining 8 years (88-68=20).

**Present Value Calculation**

Using a present value calculator or formula, we can calculate the present value of the payments for 20 years after age 68, with an interest rate of 4%:

PV = $113,600 / (1 + 0.04)^20 ≈ $44,919

Adding the present value of the remaining 8 years:

PV = $44,919 + ($61,324 / (1 + 0.04)^8) ≈ $44,919 + $34,439 ≈ $79,358

**Loss Calculation**

To calculate the loss, we need to compare the present value of the payments without "CPF Life" ($128,400) to the present value of the payments with "CPF Life" ($79,358).

Loss = ($128,400 - $79,358) / $128,400 ≈ 0.382 or 38.2%

So, the man would lose approximately 38.2% of the original amount by choosing "CPF Life".


✅ Done streaming.



In [57]:
display_answer(userPrompt,Qwen2_5_72B_model,True,"markdown");

🔁 Streaming from model: Qwen/Qwen2.5-72B-Instruct



To determine how much the man loses out by choosing "CPF Life" over not choosing it, we need to calculate the total amount of money he would receive under each option and then compare them.

### Option 1: Not Choosing CPF Life
- Monthly payment: $535
- Duration: 20 years (or 240 months)

Total amount received:
\[ \text{Total} = 535 \times 240 = 128,400 \]

### Option 2: Choosing CPF Life
- Monthly payment: $470
- Duration: 31 years (or 372 months)
- Actual lifespan: 88 years - 68 years = 20 years (or 240 months)

Total amount received in 20 years:
\[ \text{Total} = 470 \times 240 = 112,800 \]

### Comparison
To find the percentage loss, we compare the total amount received from each option:

\[ \text{Percentage Loss} = \left( \frac{\text{Amount from Option 1} - \text{Amount from Option 2}}{\text{Amount from Option 1}} \right) \times 100 \]

Substituting the values:

\[ \text{Percentage Loss} = \left( \frac{128,400 - 112,800}{128,400} \right) \times 100 \]

\[ \text{Percentage Loss} = \left( \frac{15,600}{128,400} \right) \times 100 \]

\[ \text{Percentage Loss} \approx 12.15\% \]

Therefore, by choosing "CPF Life" and living until 88 years old, the man loses out on approximately 12.15% of the total amount he would have received if he had chosen not to go for "CPF Life".


✅ Done streaming.



In [58]:
display_answer(userPrompt,DeepSeek_V33024,True,"markdown");

🔁 Streaming from model: deepseek-ai/DeepSeek-V3-0324



To determine how much the man loses out by choosing the CPF LIFE option but only living until 88 years old (20 years after his current age), we'll compare the total amounts received under both scenarios.

### **Given:**
- **Current Age:** 68 years
- **Amount Today:** \$84,552
- **Annual Interest Rate:** 4%
- **CPF LIFE Option:**
  - **Monthly Payout:** \$470
  - **Payout Duration:** 31 years (until age 99)
- **Non-CPF LIFE Option:**
  - **Monthly Payout:** \$535
  - **Payout Duration:** 20 years (until age 88)

### **Step 1: Calculate Future Value of the Lump Sum**

First, let's find out how much the lump sum (\$84,552) would grow if invested at 4% annually over 20 years (until age 88).

\[
FV = PV \times (1 + r)^n
\]

Where:
- \( PV = \$84,552 \)
- \( r = 4\% = 0.04 \)
- \( n = 20 \) years

\[
FV = 84,552 \times (1.04)^{20} \approx 84,552 \times 2.19112 \approx \$185,265.34
\]

### **Step 2: Calculate Total Payout Under CPF LIFE (Until Age 88)**

If the man chooses CPF LIFE but lives only until 88, he receives \$470 per month for 20 years (240 months):

\[
\text{Total Payout} = 470 \times 240 = \$112,800
\]

### **Step 3: Calculate Total Payout Under Non-CPF LIFE (Until Age 88)**

Under the Non-CPF LIFE option, he receives \$535 per month for 20 years (240 months):

\[
\text{Total Payout} = 535 \times 240 = \$128,400
\]

### **Step 4: Compare the Two Scenarios**

1. **CPF LIFE Payout:** \$112,800
2. **Non-CPF LIFE Payout:** \$128,400

However, the **opportunity cost** of not taking the Non-CPF LIFE option is the **future value of the lump sum (\$185,265.34)** minus the total CPF LIFE payouts (\$112,800):

\[
\text{Loss} = 185,265.34 - 112,800 = \$72,465.34
\]

But since the question asks for the **percentage loss** relative to the **Non-CPF LIFE payout**, we calculate:

\[
\% \text{Loss} = \left( \frac{128,400 - 112,800}{128,400} \right) \times 100 \approx \left( \frac{15,600}{128,400} \right) \times 100 \approx 12.15\%
\]

Alternatively, if comparing to the **future value of the lump sum**, the loss percentage would be:

\[
\% \text{Loss} = \left( \frac{185,265.34 - 112,800}{185,265.34} \right) \times 100 \approx 39.1\%
\]

**However, the most relevant comparison is between the two payout options (CPF LIFE vs. Non-CPF LIFE) for the same lifespan (20 years).** 

Thus, the man loses out by:

\[
\boxed{12.15\%}
\]

### **Final Answer:**

\boxed{12.15\%}


✅ Done streaming.



In [59]:
display_answer(userPrompt,gemma_9b_model,True,"markdown");

🔁 Streaming from model: google/gemma-2-9b-it-fast



Here's how to break down this problem to figure out the percentage loss:

**1. Calculate the total amount received from the monthly payout without CPF Life:**

* Monthly payout: $535
* Number of years: 20
* Number of months: 20 years * 12 months/year = 240 months
* Total amount received: $535/month * 240 months = $128,400

**2. Calculate the total amount received from CPF Life:**

* Monthly payout: $470
* Number of years: 31
* Number of months: 31 years * 12 months/year = 372 months
* Total amount received: $470/month * 372 months = $174,840

**3. Calculate the total amount the man would have received from his $84,552 investment without CPF Life:**

* This requires knowing the exact investment strategy and potential returns.  Since we only know about the 4% annual annuity, we can make an estimate.  

**4. Estimate the amount from the investment:**

* Assume the investment grows consistently at 4% annually for 20 years (the duration of the payout without CPF Life).
* We can use a compound interest calculator to find this.  

**5. Compare the totals:**

* Once you have the estimated total from the investment, compare it to the total from CPF Life ($174,840).

**6. Calculate the percentage loss:**

*  (Estimated investment amount - $174,840) / Estimated investment amount * 100%

**Important Considerations:**

* **Investment Growth:** The 4% annual return is an estimate.  Investment returns can fluctuate. 
* **Taxes:** Taxes on investment income may affect the final amount received.
* **Inflation:**  The purchasing power of money decreases over time due to inflation.  

**To get a more precise answer, you'd need:**

* The exact details of the investment strategy the man could have used.
*  Tax implications of both scenarios.



Let me know if you have more information about the investment strategy, and I can help you with a more accurate calculation. 



✅ Done streaming.



In [60]:
display_answer(userPrompt,openai_20b,True,"markdown");

🔁 Streaming from model: openai/gpt-oss-20b



**Answer – about a 12 % loss**

| Scenario | Monthly payment | Years | Months | Total cash received |
|----------|-----------------|-------|--------|---------------------|
| **1. No “cpf‑life”** | \$535 | 20 | 240 | \$535 × 240 = **\$128 400** |
| **2. “Cpf‑life” (but dies at 88)** | \$470 | 20 | 240 | \$470 × 240 = **\$112 800** |

*Difference in total cash flow*  
\[
\Delta = 128\,400 - 112\,800 = \$15\,600
\]

*Percentage loss relative to the better option*  

\


✅ Done streaming.



In [61]:
display_answer(userPrompt,Hermes_4_70B_model,True,"markdown");

🔁 Streaming from model: NousResearch/Hermes-4-70B



To calculate the percentage lost by choosing CPF life, we need to compare the total amount received from both options.

Option 1: Not going for CPF Life
Monthly payment = $535
Number of years = 20
Total amount = Monthly payment * (Number of years * 12)
Total amount = $535 * (20 * 12)
Total amount = $128,400

Option 2: Going for CPF Life
Monthly payment = $470
Number of years = 31
Total amount = Monthly payment * (Number of years * 12)
Total amount = $470 * (31 * 12)
Total amount = $174,720

Since he lives till 88 years old, he only receives payments for 20 years instead of 31 years.
Total amount received = Monthly payment * (20 * 12)
Total amount received = $470 * (20 * 12)
Total amount received = $112,800

Now, let's calculate the difference in the total amount received between the two options:
Difference = Total amount (Option 1) - Total amount received (Option 2)
Difference = $128,400 - $112,800
Difference = $15,600

To calculate the percentage lost, we divide the difference by the total amount received in Option 1 and multiply by 100:
Percentage lost = (Difference / Total amount (Option 1)) * 100
Percentage lost = ($15,600 / $128,400) * 100
Percentage lost = 12.15%

By choosing CPF Life, he lost out on 12.15% of the total amount he would have received if he had not gone for CPF Life.


✅ Done streaming.

